In [283]:
import pandas as pd
import numpy as np
import json
import os
import glob
import sys

In [284]:
# data = json.load(open("/Users/aahaansingh/Desktop/webscraping/CricBase/ipl_json/335982.json"))
# data

In [285]:
def match_key(data) :
    season = data["info"]["season"]
    match_number = None
    if "match_number" in data["info"]["event"] :
        match_number = str(data["info"]["event"]["match_number"])
    elif "stage" in data["info"]["event"] :
        match_number = data["info"]["event"]["stage"]
    return season, match_number

In [286]:
def match_data() :
    match_data_array = []
    match_attrs = ["season", "match_number", "city", "start_date", "winner", "batting_first", "chasing", "eliminator"]
    for filename in glob.glob(os.path.join("ipl_json", '*.json')):
        data = json.load(open(filename))
        city = None
        start_date = None
        winner = None
        eliminator = None
        season, match_number = match_key(data)
        if city in data["info"] :
            city = data["info"]["city"]
        if "dates" in data["info"] :
            if len(data["info"]["dates"]) > 0 :
                start_date = data["info"]["dates"][0]
        if "winner" in data["info"]["outcome"] :
            winner = data["info"]["outcome"]["winner"]
        elif "result" in data["info"]["outcome"] :
            winner = data["info"]["outcome"]["result"]
        if "eliminator" in data["info"]["outcome"] :
            eliminator = data["info"]["outcome"]["eliminator"]

        match_data_list = [season, match_number, city,
                   start_date, winner, data["info"]["teams"][0],
                   data["info"]["teams"][1], eliminator]
        match_data_array.append(match_data_list)
    match_df = pd.DataFrame(match_data_array, columns=match_attrs)
    return match_df
print(match_data())
        

       season match_number  city  start_date                       winner  \
0        2024           23  None  2024-04-09          Sunrisers Hyderabad   
1        2023           33  None  2023-04-23          Chennai Super Kings   
2        2009           37  None  2009-05-09          Chennai Super Kings   
3        2021           46  None  2021-10-02               Delhi Capitals   
4        2015  Qualifier 1  None  2015-05-19               Mumbai Indians   
...       ...          ...   ...         ...                          ...   
1090  2009/10           17  None  2010-03-22               Mumbai Indians   
1091  2020/21           25  None  2020-10-10  Royal Challengers Bangalore   
1092     2019           21  None  2019-04-07        Kolkata Knight Riders   
1093     2013           16  None  2013-04-13          Chennai Super Kings   
1094     2018           36  None  2018-05-05          Sunrisers Hyderabad   

                    batting_first                      chasing eliminator  

In [287]:
# def delivery_data() :
#     # will not include super over data here
#     delivery_data_array = []
#     delivery_attrs = ["season", "match_number", "over", "number", "legal_number", "batter"]

In [288]:
def player_data() :
    player_df = pd.read_csv("people.csv")
    player_match_array = []
    player_match_attrs = ["name", "player_id", "season", "match_number", "runs_scored", "fours", "sixes", 
                          "out", "balls_faced", "position", "wickets", "runs_conceded", "balls_delivered",
                          "fours_conceded", "sixes_conceded", "wides", "no_balls"]
    for filename in glob.glob(os.path.join("ipl_json", '*.json')):
        data = json.load(open(filename))
        season, match_number = match_key(data)
        for player in data["info"]["registry"]["people"] :
            player_match = [player, data["info"]["registry"]["people"][player], season, match_number,
                            None, None, None, None, None, None, None, None, None, None, None, None, None]
            player_match_array.append(player_match)
    player_match_df = pd.DataFrame(player_match_array, columns=player_match_attrs)
    return player_df, player_match_df
player_df, player_match_df = player_data()
print(player_match_df)
        

                  name player_id season match_number runs_scored fours sixes  \
0             AG Wharf  2e0fcc58   2024           23        None  None  None   
1           AK Markram  6a26221c   2024           23        None  None  None   
2          Abdul Samad  8e514b4c   2024           23        None  None  None   
3      Abhishek Sharma  f29185a1   2024           23        None  None  None   
4       Arshdeep Singh  244048f6   2024           23        None  None  None   
...                ...       ...    ...          ...         ...   ...   ...   
29980         TA Boult  a818c1be   2018           36        None  None  None   
29981        V Shankar  0994d0ae   2018           36        None  None  None   
29982        VK Sharma  a7a49df4   2018           36        None  None  None   
29983          WP Saha  fe11caa6   2018           36        None  None  None   
29984        YK Pathan  3c6ffae8   2018           36        None  None  None   

        out balls_faced position wicket

In [291]:
def delivery_data() :
    # leaving out the "legal number" field from the delivery array because it's too much work for now;
    # if I eventually figure out a simple way to figure it out then I will implement it
    delivery_array = []
    delivery_features = ["season", "match_number", "team_batting", "over", "number", "batter",
                         "bowler", "non_striker", "extras", "runs", "total_runs", "wickets"]
    wicket_array = []
    wicket_features = ["season", "match_number", "team_batting", "over", "number", "player_out", "type"]
    extra_array = []
    extra_features = ["season", "match_number", "team_batting", "over", "number", "byes", "legbyes", "noballs", "penalty", "wides"]
    fielder_wicket_array = []
    fielder_wicket_features = ["season", "match_number", "team_batting", "over", "number", "id"] # not name, which isn't necessarily unique
    for filename in glob.glob(os.path.join("ipl_json", '*.json')):
        data = json.load(open(filename))
        season, match_number = match_key(data)
        registry = data["info"]["registry"]["people"]
        for innings_data in data["innings"] :
            team_batting = innings_data["team"]
            for over_data in innings_data["overs"] :
                over = over_data["over"]
                for number, delivery_data in enumerate(over_data["deliveries"]) :
                    batter_name = delivery_data["batter"]
                    batter_id = registry[batter_name]
                    bowler_name = delivery_data["bowler"]
                    bowler_id = registry[bowler_name]
                    non_striker_name = delivery_data["non_striker"]
                    non_striker_id = registry[non_striker_name]
                    runs = delivery_data["runs"]["batter"]
                    total_runs = delivery_data["runs"]["total"]
                    wickets = 0
                    extras = delivery_data["runs"]["extras"]

                    if "wickets" in delivery_data :
                        wickets = len(delivery_data["wickets"])
                        for wicket_data in delivery_data["wickets"] :
                            wicket_type = wicket_data["kind"]
                            player_out_name = wicket_data["player_out"]
                            player_out_id = registry[player_out_name]
                            wicket_list = [season, match_number, team_batting, over, number, player_out_id, wicket_type]
                            wicket_array.append(wicket_list)
                            if "fielders" in wicket_data:
                                for fielder_data in wicket_data["fielders"] :
                                    fielder_name = fielder_data["name"]
                                    fielder_id = registry[fielder_name]
                                    fielder_wicket_list = [season, match_number, team_batting, over, number, fielder_id]
                                    fielder_wicket_array.append(fielder_wicket_list)
                    
                    if "extras" in delivery_data :
                        extras_data = delivery_data["extras"]
                        byes, legbyes, noballs, penalty, wides = 0, 0, 0, 0, 0
                        if "byes" in extras_data :
                            byes = extras_data["byes"]
                        if "legbyes" in extras_data :
                            byes = extras_data["legbyes"]
                        if "noballs" in extras_data :
                            byes = extras_data["noballs"]
                        if "penalty" in extras_data :
                            byes = extras_data["penalty"]
                        if "wides" in extras_data :
                            byes = extras_data["wides"]
                        extra_list = [season, match_number, team_batting, over, number, byes, 
                                       legbyes, noballs, penalty, wides]
                        extra_array.append(extra_list)
                    delivery_list = [season, match_number, team_batting, over, number, batter_id, 
                                     bowler_id, non_striker_id, extras, runs, total_runs, wickets]
                    delivery_array.append(delivery_list)
    delivery_df = pd.DataFrame(delivery_array, columns=delivery_features)
    wicket_df = pd.DataFrame(wicket_array, columns=wicket_features)
    extra_df = pd.DataFrame(extra_array, columns=extra_features)
    fielder_wicket_df = pd.DataFrame(fielder_wicket_array, columns=fielder_wicket_features)
    return delivery_df, wicket_df, extra_df, fielder_wicket_df

delivery_df, wicket_df, extra_df, fielder_wicket_df = delivery_data()
print(fielder_wicket_df)
                    
                        



                            


     season match_number         team_batting  over  number        id
0      2024           23  Sunrisers Hyderabad     3       1  0a476045
1      2024           23  Sunrisers Hyderabad     3       3  800d2d97
2      2024           23  Sunrisers Hyderabad     4       6  26989d80
3      2024           23  Sunrisers Hyderabad     9       3  800d2d97
4      2024           23  Sunrisers Hyderabad    13       0  e94915e6
...     ...          ...                  ...   ...     ...       ...
9855   2013           16  Chennai Super Kings    12       1  00ea847a
9856   2013           16  Chennai Super Kings    18       0  7daedf2f
9857   2018           36     Delhi Daredevils    10       0  dcf81436
9858   2018           36     Delhi Daredevils    15       0  0a476045
9859   2018           36  Sunrisers Hyderabad    17       0  8b3e9c7c

[9860 rows x 6 columns]


In [ ]:
def player_stats() :
    for player_row in player_match_df :
        deliveries_faced_df = delivery_df.loc[(delivery_df["batter"] == player_row["player_id"])
                                              and (delivery_df["season"] == player_row["season"])
                                              and (delivery_df["match_number"] == player_row["match_number"])]
        deliveries_bowled_df = delivery_df.loc[(delivery_df["bowler"] == player_row["player_id"])
                                              and (delivery_df["season"] == player_row["season"])
                                              and (delivery_df["match_number"] == player_row["match_number"])]
        player_row[""]